In [1]:
from keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, AveragePooling2D, Dropout
from keras.layers import UpSampling2D
from keras.models import Model

from keras.utils import to_categorical
from keras.datasets import cifar10

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import pickle, os, zipfile, glob
from tqdm import tqdm
from keras import backend as K
from keras.utils.training_utils import multi_gpu_model
import time

/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)

In [7]:
!pwd

/home/seung/data/temp/Pseudo-Label-Keras


In [9]:
import os
myhost = os.uname()[1]
print(myhost)

seung-DevBox


In [2]:
to_Train = True
#nb_labeled_data = 500
nb_epoch = 10

In [ ]:
import sys
print(sys.executable)

In [3]:
if not os.path.exists("result_supervised"):
        os.mkdir("result_supervised")

In [4]:
def basic_conv_block(input, chs, rep):
    x = input
    for i in range(rep):
        x = Conv2D(chs, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
    return x

def create_cnn2():
    from keras.applications import MobileNet
    net = MobileNet(input_shape=(128,128,3), weights=None, include_top=False)
    # upsampling(32->128)
    input = Input((32,32,3))
    x = UpSampling2D(4)(input)
    x = net(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation="softmax")(x)

    model = Model(input, x)
    #model.summary()
    return model

In [5]:
def train(n_labeled_data, n_epoch=1):
    model = create_cnn2()
    model.summary()
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    K.image_data_format()
    
    indices = np.arange(X_train.shape[0])
    np.random.shuffle(indices)

    y_test_true = np.ravel(y_test)
    X_train = X_train[indices[:n_labeled_data]] / 255.0
    X_test = X_test / 255.0
    y_train = to_categorical(y_train[indices[:n_labeled_data]], 10)
    y_test = to_categorical(y_test, 10)
    #model = multi_gpu_model(model, gpus=2)
    model.compile("adam", loss="categorical_crossentropy", metrics=["acc"])

    
    t0 = time.time()
    hist = model.fit(X_train, y_train, batch_size=min(n_labeled_data, 256),
                     validation_data=(X_test, y_test), epochs=n_epoch, verbose=1).history
    elapsed = time.time() - t0
    print('elapsed = {}... {} per epoch'.format(elapsed, elapsed/n_epoch))
    fn1 = "result_supervised/history_{}.dat".format(n_labeled_data)
    
    with open(fn1,  "wb") as fp:
        pickle.dump(hist, fp)
        print(fn1)

    fnmodel = "result_supervised/model_{}_{}.h5".format('MobileNet', n_labeled_data)
    model.save(fnmodel)
    print('Saved model..... {}'.format(fnmodel))
    return model

In [6]:
vtraining = [500, 1000, 5000, 10000]
vtraining  = [500,  5000, 10000]
vtraining  = [500,]
for ntr in vtraining :
    print('{} ---------------------'.format(ntr))
    model1 = train(ntr, n_epoch=nb_epoch)    

W0827 23:02:38.752733 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 23:02:38.764742 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 23:02:38.768810 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 23:02:38.787136 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v

500 ---------------------


W0827 23:02:41.380635 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0827 23:02:43.317571 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 128, 3)       0         
_________________________________________________________________
mobilenet_1.00_128 (Model)   (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 3,239,114
Trainable params: 3,217,226
Non-trainable params: 21,888
_________________________________________________________________


W0827 23:02:45.182873 139852474005248 deprecation_wrapper.py:119] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0827 23:02:45.295685 139852474005248 deprecation.py:323] From /home/seung/.venv/py3Keras/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 500 samples, validate on 10000 samples
Epoch 1/10
500/500 [==============================] - 113s 226ms/step - loss: 2.4775 - acc: 0.1120 - val_loss: 9.1515 - val_acc: 0.1000
Epoch 2/10
500/500 [==============================] - 109s 218ms/step - loss: 2.5189 - acc: 0.1780 - val_loss: 5.7079 - val_acc: 0.1055
Epoch 3/10
500/500 [==============================] - 109s 217ms/step - loss: 2.3252 - acc: 0.1960 - val_loss: 3.9991 - val_acc: 0.1295
Epoch 4/10
500/500 [==============================] - 109s 218ms/step - loss: 2.1231 - acc: 0.2520 - val_loss: 3.9373 - val_acc: 0.1114
Epoch 5/10
500/500 [==============================] - 109s 217ms/step - loss: 2.0107 - acc: 0.3660 - val_loss: 4.0598 - val_acc: 0.1302
Epoch 6/10
500/500 [==============================] - 109s 218ms/step - loss: 1.8233 - acc: 0.4320 - val_loss: 3.3406 - val_acc: 0.1360
Epoch 7/10
500/500 [==============================] - 109s 217ms/step - loss: 1.5381 - acc: 0.5660 - val_loss: 3.1952 - val_acc: 0.1412
